In [1]:
%pip install psycopg2
%pip install configparser
%pip install configparser

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd
import configparser

# Read and import environement variables from cred.ini
config = configparser.ConfigParser()
config.sections()

config.read('cred.ini')


['cred.ini']

In [3]:
#connect to source database on the cloud using cred.ini source variables
db_source_conn = psycopg2.connect(
    database=config['SOURCE']['database'], 
    user=config['SOURCE']['user'], 
    password=config['SOURCE']['password'], 
    host=config['SOURCE']['host'], 
    port= config['SOURCE']['port']
    )

db_source_cursor = db_source_conn.cursor()

In [4]:
# Query to get all the data from the public.User table
db_source_cursor.execute(
    'select * from public."User"'
    )

# fetch all data 
result_source_hive_user = db_source_cursor.fetchall()

colnames = [desc[0] for desc in db_source_cursor.description]

df_user = pd.DataFrame(result_source_hive_user, columns=colnames)
df_user.to_csv('./New Data/user.csv',sep=';',index=False)

In [5]:
# Query to get all the data from the public.Hive_Data table
db_source_cursor.execute(
    'select * from public."HiveData"'
    )

# fetch all data 
result_source_hive_data = db_source_cursor.fetchall()

colnames = [desc[0] for desc in db_source_cursor.description]

df_hive_data = pd.DataFrame(result_source_hive_data, columns=colnames)
df_hive_data.to_csv('./New Data/hive_data.csv',sep=';',index=False)

In [6]:
# Query to get all the data from the public.Hive table
db_source_cursor.execute(
    'select * from public."Hive"'
    )

# fetch all data 
result_source_hive = db_source_cursor.fetchall()

colnames = [desc[0] for desc in db_source_cursor.description]

df_hive = pd.DataFrame(result_source_hive, columns=colnames)
df_hive.to_csv('./New Data/hive.csv',sep=';',index=False)

In [7]:
# Query to get all the data from the public.Hive table
db_source_cursor.execute(
    'select * from public."Session"'
    )

# fetch all data 
result_source_session = db_source_cursor.fetchall()

colnames = [desc[0] for desc in db_source_cursor.description]

df_session = pd.DataFrame(result_source_session, columns=colnames)
df_session.to_csv('./New Data/session.csv',sep=';',index=False)

In [8]:
df_session['sess']

0    {'cookie': {'originalMaxAge': 7200000, 'expire...
Name: sess, dtype: object

In [9]:
db_source_conn.close()

In [10]:
db_target_conn = psycopg2.connect(
    database=config['TARGET']['database'], 
    user=config['TARGET']['user'], 
    password= "I'llbethere4U" ,
    host=config['TARGET']['host'], 
    port= config['TARGET']['port'],
    options=f"-c search_path=dbo,{config['TARGET']['schema']}"
    )

db_target_cursor = db_target_conn.cursor()

In [11]:
file_path = "queries.sql"

try:
    # Lire les requêtes SQL depuis le fichier
    with open(file_path, "r") as file:
        queries = file.read()

    # Exécuter les requêtes SQL
    db_target_cursor.execute(queries)
    db_target_conn.commit()
    print("Requêtes exécutées avec succès.")
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Erreur: {error}")
    db_target_conn.rollback()
finally:
    # Fermer le curseur et la connexion
    db_target_cursor.close()


Requêtes exécutées avec succès.


In [12]:
def copy_from_csv(conn, table_name, csv_file_path):
    with conn.cursor() as cursor:
        try:
            with open(csv_file_path, 'r') as f:
                cursor.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER DELIMITER ';'", f)
            conn.commit()
            print(f"Data copied successfully from {csv_file_path} to {table_name}.")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error: {error}")
            conn.rollback()

In [13]:
db_target_cursor = db_target_conn.cursor()

In [14]:
'oLkQwZp-fJRDa-ePxIY2QqIs2G6h1SAA'

a = {
    'cookie': {'originalMaxAge': 7200000, 'expires': '2024-05-24T15:33:24.316Z', 'secure': False, 'httpOnly': True, 'path': '/', 'sameSite': 'lax'}, 'userId': '01c02394-8662-442b-8d20-8af9ee75ab9d'}
b = '2024-05-24 15:33:25'


In [15]:
copy_from_csv(db_target_conn, '"user"', r".\New Data\user.csv")

Data copied successfully from .\New Data\user.csv to "user".


In [16]:
copy_from_csv(db_target_conn, '"hive"', r".\New Data\hive.csv")

Data copied successfully from .\New Data\hive.csv to "hive".


In [17]:
copy_from_csv(db_target_conn, '"hive_data"', r".\New Data\hive_data.csv")


Data copied successfully from .\New Data\hive_data.csv to "hive_data".


In [18]:
copy_from_csv(db_target_conn, '"session"', r".\New Data\session.csv")


Data copied successfully from .\New Data\session.csv to "session".
